## PEAK 3 Y24 phosphorylation

In [4]:
from models.ModelBuilder import ModelBuilder
from models.Reaction import Reaction
from models.ReactionArchtype import ReactionArchtype
from models.ArchtypeCollections import *

model = ModelBuilder('PEAK3 Y24 Phosphorylation')
model.add_reaction(Reaction(michaelis_menten, ('PEAK3_Y24',), ('PEAK3_Y24P',)))
model.add_reaction(Reaction(michaelis_menten, ('PEAK3_Y24P',), ('PEAK3_Y24',)))

model.simulate(0, 100, 100)
model.plot()


RuntimeError: SBML document must begin with an XML declaration or an SBML declaration